### Openapi
- 데이터를 제공하는 주소와 요청 시 필요한 데이터들을 요청 메시지에 추가하여 요청(request)
- 필요한 데이터들과 주소가 명확하다면 응답 메시지 반환
- 서버에게 요청을 보내는 라이브러리 : requests
    - requests에 내장된 get() 함수를 이용하여 요청을 보내고 응답 메시지를 받는다. 

In [ ]:
import requests

In [ ]:
url = 'https://apis.data.go.kr/B190001/salesPolicy'
service_key = 'dtbWOdJ/Cz5HE0DGLU+CRPe7pOW0NIQBUcGEqsHZaTRiYCI/5+zugwzQjcvuId7NPdg6rUiW+ft3fm7yqyD4pw=='
sub_url = '/paper'

params = {
    'serviceKey' : service_key, 
    'pageNo' : 1, 
    'numOfRows' : 10
}

res = requests.get(url+sub_url, params)

In [ ]:
res

In [ ]:
type(res.content)

In [ ]:
import pandas as pd

In [ ]:
pd.DataFrame(res.content)

In [ ]:
# bytes 타입의 데이터는 DataFrame 변환이 불가능
# json 형태의 데이터인 경우 -> json 라이브러리 호출
# json 라이브러리 안에 loads() 함수를 이용하여 데이터의 타입을 변환 
import json 
res_data = json.loads(res.content)

In [ ]:
type(res_data)

In [18]:
# res_data에는 데이터셋 뿐만이 아니라 요청에 대한 응답 결과들이 같이 포함
# 원하는 데이터셋만 따로 추출하여 데이터프레임화 
df = pd.DataFrame(res_data['data'])
df2 = df.copy()

In [19]:
# 데이터프레임에서 각각의 컬럼의 이름들은 api을 제공하는 사이트에서 확인하여 이름을 변경 
# df.columns -> 전체 컬럼의 이름을 변경 (class의 속성의 값을 변경)
# df.rename() -> 함수를 이용하여 컬럼의 이름을 변경 (class의 메서드를 이용하여 내부의 데이터를 변경)
df.columns = ['법인할인구매가능여부', '법인단순구매가능여부', '기준일자', '할인정책적용시작일자', 
 '할인정책종료일자', '할인율', '일간구매제한금액', '예외정책여부', '최대할인제한금액', 
 '최대환전제한금액', '월간구매제한금액', '제공기관코드', '사용처지역코드', '연간구매제한금액']

In [ ]:
df.info()

In [22]:
cols = df2.columns
new_cols = ['법인할인구매가능여부', '법인단순구매가능여부', '기준일자', '할인정책적용시작일자', 
 '할인정책종료일자', '할인율', '일간구매제한금액', '예외정책여부', '최대할인제한금액', 
 '최대환전제한금액', '월간구매제한금액', '제공기관코드', '사용처지역코드', '연간구매제한금액']

In [23]:
cols_dict = {}
for old, new in zip(cols, new_cols):
    cols_dict[old] = new

In [ ]:
cols_dict

In [ ]:
# rename 함수를 이용하여 컬럼의 이름을 변경 
# rename 함수의 장점 : 인덱스나 컬럼이나 모두 변경이 가능, 
# columns 속성의 값을 변환할 때는 확인 불가능하지만 rename은 확인 하고 변환이 가능
# inplace 매개변수의 값을 True로 변경하여 데이터를 변환
df2.rename(columns = cols_dict, inplace=True)

In [ ]:
# api에서 수집한 데이터를 저장 
# 파일로 저장 ( csv, excel, json, xml 파일의 형태를 지정하여 저장 ) -> to_xxxx() 함수
# to -> 일반적으로 데이터의 타입을 변경할때 자주 사용하는 키워드 
# to_csv -> csv 타입으로 변경(파일로 저장 -> 저장하는 위치)
# excel 파일로 저장 -> to_excel()
# list 형태로 변환 -> to_list()
# dict 형태로 변환 -> to_dict()
df.to_csv( "./test.csv", index = False, )
# mode 매개변수 -> 인자를 'a' -> append 모드 -> 기존의 파일에 데이터를 추가하는 형태 

In [ ]:
import os 
# 특정 경로에 파일이 존재하는가?

# 존재유무판단-> is 
# 파일 -> file

if os.path.isfile("./test.csv"):
    # 특정 경로에 파일이 존재하는 경우
    # 해당 파일에 데이터를 추가 
    # 데이터프레임에서 인덱스 저장x 헤더(머리 부분의 데이터 -> 컬럼명) 저장x
    df.to_csv('./test.csv', index=False, mode='a', header=False)
else:
    # 파일이 존재하지 않는 경우 
    # 파일을 새로 생성하여 헤더는 저장o, 인덱스는 저장x
    df.to_csv('./test.csv', index=False)
# mode 매개변수 
    # 'w'(기본값) : 기존의 파일이 존재한다면 대체, 없으면 생성
    # 'a' : 기존의 파일에 데이터를 추가 
    # 'x' : 기존이 파일이 존재한다면 대입하지 않겠다.(기존의 파일을 보존)

In [ ]:
# 자체적인 DataBase에 저장 
# 1. 각 행의 데이터들을 insert query 문을 이용하여 대입 
# 2. sqlalchemy 라이브러리를 이용하여 DBserver와의 연결을 하고 to_sql()을 이용하여 
#    데이터프레임 전체를 테이블에 대입
#       (테이블이 존재하는 경우 -> 교체, 추가 / 테이블이 존재하지 않을때는 테이블생성) 